# Part A: Data Processing


## Python Notebook for Part A (Data Processing)

In this section, I used a Python notebook as the primary tool to implement the data processing tasks. The Pandas library was utilized to manipulate data frames, including checking for missing values, handling erroneous values, calculating derived columns, and performing joins and merges on multiple data frames. The final results were saved to an Excel file.


In [12]:
# Loading data to DataFrame Pandas
import pandas as pd

file_path = 'Data.xlsx'
transactions_df = pd.read_excel(file_path, sheet_name='Data Transactions')
commission_df = pd.read_excel(file_path, sheet_name='Data Commission')
user_info_df = pd.read_excel(file_path, sheet_name='Data User_Info')

print(transactions_df.head())
print(len(transactions_df))
print(commission_df.head()) 
print(len(commission_df))
print(user_info_df.head())
print(len(user_info_df))



    User_id    order_id  Amount  Merchant_id Purchase_status  Revenue  \
0  21269588  4169517626  10,000           13             NaN      300   
1  28097592  4170276686  20,000           13             NaN      600   
2  47435144  4166729310  10,000           12             NaN      200   
3  29080935  4174460303  10,000           13             NaN      300   
4  14591075  4168216749  10,000           12             NaN      200   

  ConvertedDate  Month Day_of_Week  Gender       Age Location First_tran_date  \
0    2020-01-01      1   Wednesday  Female       >37   Ha Noi      2018-12-11   
1    2020-01-01      1   Wednesday  Female       >37   Ha Noi      2019-12-30   
2    2020-01-01      1   Wednesday  Female  18_to_22   Ha Noi      2019-11-11   
3    2020-01-01      1   Wednesday  Female  18_to_22   Ha Noi      2019-10-24   
4    2020-01-01      1   Wednesday  Female  18_to_22    Other      2019-12-28   

  Type_user  
0   Current  
1   Current  
2   Current  
3   Current  
4   

### Task 1: Revenue and Revenue in Jan



In [4]:
revenue_list = []
rate_by_merchant_id = {}

for i in range(len(commission_df)):
    rate_by_merchant_id[commission_df['Merchant_id'][i]] = int(commission_df['Rate_pct'][i])/100

for i in range(len(transactions_df)):
    revenue = int(int(transactions_df['Amount'][i].replace(",", "")) * rate_by_merchant_id[transactions_df['Merchant_id'][i]])
    revenue_list.append(revenue)
# Calculate the revenue for each transaction
transactions_df['Revenue'] = revenue_list
# Dataframe after caculated Revenue value
print(len(transactions_df))
# Calculate MoMo's total revenue in January 2020
total_revenue_jan_2020 = 0
for i in range(len(transactions_df)):
    if (transactions_df['Date'][i] >= '2020-01-01') and (transactions_df['Date'][i] <= '2020-01-31'):
        total_revenue_jan_2020 += transactions_df['Revenue'][i]
print(f"Total revenue for January 2020: {total_revenue_jan_2020}")

13495
Total revenue for January 2020: 1409827


### Task 2: Most profitable month?

In [5]:
#Columns Date have 2 different format of the date
invalid_date_count = 0
for i in range(len(transactions_df)):
    try:
        transactions_df.loc[i, 'ConvertedDate'] = pd.to_datetime(transactions_df.loc[i, 'Date'], format='%Y-%m-%d')
    except:
        try:
            transactions_df.loc[i, 'ConvertedDate'] = pd.to_datetime(transactions_df.loc[i, 'Date'], format='%d/%m/%Y')
        except:
            invalid_date_count += 1
            print(transactions_df.loc[i, 'Date'])

print("Created ConvertedDate column")
print(f"Have {(invalid_date_count)} invalid date in ConvertedDate column")
transactions_df.drop('Date', axis=1, inplace=True)
# Extract the month from valid dates
transactions_df['Month'] = transactions_df['ConvertedDate'].dt.month


# Find the maximum revenue
# Group by 'Month' and sum revenue
monthly_revenue = transactions_df.groupby('Month')['Revenue'].sum().reset_index()

max_revenue = monthly_revenue['Revenue'].max()

# Get the corresponding month using boolean indexing
max_revenue_month = monthly_revenue[monthly_revenue['Revenue'] == max_revenue]['Month'].values[0]

# Print the month with the maximum revenue
print(f"The month with the maximum revenue is: {max_revenue_month} with Revenue = {max_revenue}")


Created ConvertedDate column
Have 0 invalid date in ConvertedDate column
The month with the maximum revenue is: 9 with Revenue = 1702200


### Task 3: Day of week (most and least money):

In [6]:

transactions_df['Day_of_Week'] = transactions_df['ConvertedDate'].dt.day_name()
average_revenue_by_day = transactions_df.groupby('Day_of_Week')['Revenue'].mean().reset_index()

max_revenue = average_revenue_by_day['Revenue'].max()
min_revenue = average_revenue_by_day['Revenue'].min()
# Get the corresponding month using boolean indexing
max_revenue_day = average_revenue_by_day[average_revenue_by_day['Revenue'] == max_revenue]['Day_of_Week'].values[0]
min_revenue_day = average_revenue_by_day[average_revenue_by_day['Revenue'] == min_revenue]['Day_of_Week'].values[0]
# Print the month with the maximum revenue
print(f"The day with the maximum revenue is {max_revenue_day} with Reveue = {max_revenue}")
print(f"The day with the min revenue is {min_revenue_day} with Reveue = {min_revenue}")


print(len(transactions_df))

The day with the maximum revenue is Wednesday with Reveue = 1482.535074987905
The day with the min revenue is Monday with Reveue = 1304.437564499484
13495


### Task 4: The total number of new users in December 2020.


In [7]:
# Remove duplicate rows based on 'User_id', keeping the last occurrence
user_info_df_unique = user_info_df.drop_duplicates(subset=['User_id'], keep='last')

# Merge the dataframes
transactions_df = transactions_df.merge(user_info_df_unique[['User_id', 'Gender', 'Age', 'Location', 'First_tran_date']], on='User_id', how='inner')


#Ensure 'First_tran_date' column is in datetime format
invalid_date_count = 0
for i in range(len(transactions_df)):
    try:
        transactions_df.loc[i, 'First_tran_date'] = pd.to_datetime(transactions_df.loc[i, 'First_tran_date'], format='%Y-%m-%d')
    except:
        invalid_year = transactions_df.loc[i, 'First_tran_date'][0:4]
        if invalid_year [:2] == '99':
            valid_year = invalid_year.replace('99','20')
        else:
            valid_year = invalid_year.replace('30','20')
        valid_date = valid_year + transactions_df.loc[i, 'First_tran_date'][4:]
        # print('Wrong Date: ', invalid_year)
        # print('Fixed Date: ',valid_date)
        try:
            transactions_df.loc[i, 'First_tran_date'] = pd.to_datetime(valid_date, format='%Y-%m-%d')
        except:
            invalid_date_count += 1
            print('Wrong Date: ',transactions_df.loc[i, 'First_tran_date'])

print(f"Have {(invalid_date_count)} invalid date")
# Check missing value in the First_tran_date column
print(str(transactions_df['First_tran_date'].isnull().sum())+ ' missing value in First_tran_date columns')



Have 0 invalid date
0 missing value in First_tran_date columns


In [8]:
# Create Type_user columns
transactions_df['Type_user'] = transactions_df.apply(
    lambda row: 'New' if row['ConvertedDate'].year == row['First_tran_date'].year and row['ConvertedDate'].month == row['First_tran_date'].month else 'Current',
    axis=1)
dec_transactions = transactions_df[(transactions_df['ConvertedDate'] >= '2020-12-01') & (transactions_df['ConvertedDate'] <= '2020-12-31')]

# Find the unique number of ner user in December 2020
new_users_dec_2020 = dec_transactions[dec_transactions['Type_user'] == 'New']['User_id'].nunique()
print('Total new user in December 2020 is:', new_users_dec_2020)

Total new user in December 2020 is: 76


In [9]:
# Function to standardize the Gender column
def standardize_gender(gender):
    if gender.lower() in ['f', 'female', 'nữ']:
        return 'Female'
    elif gender.lower() in ['m', 'male', 'nam']:
        return 'Male'
    else:
        return 'Unknown'

# Function to standardize the Location column
def standardize_location(location):
    if location in ['HN', 'Ha Noi']:
        return 'Ha Noi'
    elif location in ['HCMC', 'Ho Chi Minh City']:
        return 'Ho Chi Minh City'
    elif location in ['Other', 'Other Cities', 'Unknown']:
        return 'Other'
    else:
        return location

# Applying the functions to the dataframe
transactions_df['Gender'] = transactions_df['Gender'].apply(standardize_gender)
transactions_df['Location'] = transactions_df['Location'].apply(standardize_location)


In [11]:
print(str(transactions_df['Gender'].isnull().sum())+ ' missing value in Gender columns')
print(str(transactions_df['Location'].isnull().sum())+ ' missing value in Location columns')

0 missing value in Gender columns
0 missing value in Location columns


In [10]:
# Save processed Data
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    transactions_df.to_excel(writer, sheet_name='Data Transactions')